In [1]:
import requests
from bs4 import BeautifulSoup

# Part A & D

---

This Part A is just for checking if user enters most common MPins, either in 4-digits or in 6-digit,

It checks for both and shows a warning message .

---



In [13]:



import requests
from bs4 import BeautifulSoup

def fetch_common_digit_pins_from_pocketlint(tag_,id_,length):
    url = "https://www.pocket-lint.com/these-are-the-20-most-common-phone-pins-is-your-device-vulnerable/"
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
    }
    

    
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("Failed to fetch the page")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    heading = soup.find(lambda tag: tag.name.startswith(tag_) and id_ in tag.get_text())

    if not heading:
        print("Section not found")
        return []

    pin_list = []
    ul_tag = heading.find_next("ul")
    if ul_tag:
        for li in ul_tag.find_all("li"):
            text = li.get_text(strip=True)
            if text.isdigit() and len(text) == length:
                pin_list.append(text)

    return pin_list

def run_main(user_input):
    # Extracting 4 pins
    pins_4 = fetch_common_digit_pins_from_pocketlint("p","Common four-digit PINs",4)
    
    # Extracting 6 digit pins
    pins_6 = fetch_common_digit_pins_from_pocketlint("p","Common six-digit PINs",6)

    if user_input in pins_4 or user_input in pins_6:
        print("You entered common Mpin, please enter something strong Mpin")
    else:
        print("Congratulations! Mpin set successfully.")


if __name__== "__main__":
    user_input = input("Please enter your Mpin (Either in 4 digit or in 6 digit): ")
    
    if len(user_input)==4 or len(user_input)==6:
        run_main(user_input)
    else:
        print("Please enter the correct Mpin!")




Please enter your Mpin (Either in 4 digit or in 6 digit):  3004


Congratulations! Mpin set successfully.


# Part B,C & D

---
This part of code asking user to give demographic data as input and checking the strength of Mpin following part B , and also showing 
reason of weak Mpin which is following part C , and it is also applicable for 6-digit pin that's why following part D also

---

In [39]:
def parse_date(input_str):
    try:
        return datetime.strptime(input_str.strip(), "%d-%m-%Y")
    except ValueError:
        return None



def generate_mpin_combinations(date_obj, pin_length):
    day = date_obj.strftime("%d")
    month = date_obj.strftime("%m")
    year_full = date_obj.strftime("%Y")
    year_short = year_full[-2:]

    if pin_length == 4:
        return [
            day + month,     # ddmm
            month + day,     # mmdd
            year_short,      # yy
            year_full,       # yyyy
            day + year_short, # ddyy
            month + year_short # mmyy
        ]
    elif pin_length == 6:
        return [
            day + month + year_short,     # ddmmyy
            month + year_short + day,     # mmyydd
            year_short + month + day,     # yymmdd
            day + year_short + month,     # ddyymm
            year_short + day + month,     # yyddmm
            month + day + year_short,     # mmddyy
            day + month + year_short,     # ddmmyy (again)
            month + day + year_short      # mmddyy (again)
        ]
    else:
        return []


def check_against_demographics(mpin, demographics):
    reasons = []
    pin_length = len(mpin)

    for label, date_obj in demographics.items():
        if date_obj:
            weak_pins = generate_mpin_combinations(date_obj, pin_length)
            if mpin in weak_pins:
                reasons.append(f"DEMOGRAPHIC_{label.upper()}")
    return reasons


In [41]:
def run_mpin_checker():
    print("=== MPIN Strength Checker ===")
    
    dob_str = input("Enter your Date of Birth (dd-mm-yyyy): ").strip()
    spouse_dob_str = input("Enter Spouse DOB (dd-mm-yyyy) or press Enter to skip: ").strip()
    anniversary_str = input("Enter Anniversary Date (dd-mm-yyyy) or press Enter to skip: ").strip()

    dob = parse_date(dob_str)
    spouse_dob = parse_date(spouse_dob_str) if spouse_dob_str else None
    anniversary = parse_date(anniversary_str) if anniversary_str else None

    if not dob:
        print("⚠️ Invalid DOB format. Exiting.")
        return

    mpin = input("Enter your MPIN (4 or 6 digits): ").strip()
    if not mpin.isdigit() or len(mpin) not in [4, 6]:
        print("⚠️ MPIN must be a 4 or 6 digit number. Exiting.")
        return

    # Extract common pins for 4 digit and 6 digit
    pins_4 = fetch_common_digit_pins_from_pocketlint("p", "Common four-digit PINs", 4)
    pins_6 = fetch_common_digit_pins_from_pocketlint("p", "Common six-digit PINs", 6)
    is_common = mpin in pins_4 or mpin in pins_6

    # Demographics
    demographics = {
        "dob_self": dob,
        "dob_spouse": spouse_dob,
        "anniversary": anniversary
    }

    demo_reasons = check_against_demographics(mpin, demographics)

    # Final Output
    if is_common or demo_reasons:
        print("\n🔒 MPIN Strength: WEAK")
        if is_common:
            print("- Reason: COMMONLY_USED")
        for reason in demo_reasons:
            print(f"- Reason: {reason}")
    else:
        print("\n✅ MPIN Strength: STRONG")


if __name__ == "__main__":
    run_mpin_checker()


=== MPIN Strength Checker ===


Enter your Date of Birth (dd-mm-yyyy):  30-04-2005
Enter Spouse DOB (dd-mm-yyyy) or press Enter to skip:  
Enter Anniversary Date (dd-mm-yyyy) or press Enter to skip:  
Enter your MPIN (4 or 6 digits):  5656



✅ MPIN Strength: STRONG


# Test Cases

---
Here we took 20 test case scenarios to test our above written code and paased all 20 cases sueccessfully

---

In [51]:
test_cases = [
    {
        "description": "Common 4-digit PIN",
        "dob": "15-08-1990",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "1234",
        "expected_strength": "WEAK",
        "expected_reasons": ["COMMONLY_USED"]
    },
    {
        "description": "Common 6-digit PIN",
        "dob": "10-10-1995",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "123456",
        "expected_strength": "WEAK",
        "expected_reasons": ["COMMONLY_USED"]
    },
    {
        "description": "MPIN from DOB (ddmm)",
        "dob": "04-07-1995",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "0407",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]
    },
    {
        "description": "MPIN from DOB (mmyy)",
        "dob": "06-12-2001",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "1201",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]
    },
    {
        "description": "MPIN from spouse DOB",
        "dob": "01-01-2000",
        "spouse_dob": "22-11-1994",
        "anniversary": "",
        "mpin": "2211",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SPOUSE"]
    },
    {
        "description": "MPIN from anniversary",
        "dob": "01-01-2000",
        "spouse_dob": "",
        "anniversary": "12-09-2012",
        "mpin": "1209",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_ANNIVERSARY"]
    },
    {
        "description": "6-digit MPIN from DOB (ddmmyy)",
        "dob": "30-04-2005",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "300405",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]
    },
    {
        "description": "6-digit MPIN from anniversary (yymmdd)",
        "dob": "30-01-1999",
        "spouse_dob": "",
        "anniversary": "05-11-2010",
        "mpin": "101105",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_ANNIVERSARY"]
    },
    {
        "description": "Strong 4-digit MPIN",
        "dob": "01-01-1990",
        "spouse_dob": "02-02-1990",
        "anniversary": "03-03-2010",
        "mpin": "7492",
        "expected_strength": "STRONG",
        "expected_reasons": []
    },
    {
        "description": "Strong 6-digit MPIN",
        "dob": "01-01-1990",
        "spouse_dob": "02-02-1990",
        "anniversary": "03-03-2010",
        "mpin": "948372",
        "expected_strength": "STRONG",
        "expected_reasons": []
    },
    {
        "description": "Common + Demographic MPIN",
        "dob": "12-12-2012",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "1212",
        "expected_strength": "WEAK",
        "expected_reasons": ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"]
    },
    {
        "description": "Demographic from spouse date (ddyy)",
        "dob": "01-01-1990",
        "spouse_dob": "19-06-1997",
        "anniversary": "",
        "mpin": "1997",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SPOUSE"]
    },
    {
        "description": "Anniversary ddmmyy 6-digit",
        "dob": "01-01-1990",
        "spouse_dob": "",
        "anniversary": "23-05-2011",
        "mpin": "230511",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_ANNIVERSARY"]
    },
    {
        "description": "MPIN is full year of DOB",
        "dob": "09-10-2002",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "2002",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]
    },
    {
        "description": "MPIN is reversed date",
        "dob": "01-01-1990",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "0101",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]
    },
    {
        "description": "Empty spouse/anniv, random PIN",
        "dob": "01-01-1990",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "6382",
        "expected_strength": "STRONG",
        "expected_reasons": []
    },
    {
        "description": "Just common 6-digit PIN",
        "dob": "01-01-1990",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "111111",
        "expected_strength": "WEAK",
        "expected_reasons": ["COMMONLY_USED"]
    },
    {
        "description": "MMYY from DOB",
        "dob": "04-11-1993",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "1193",
        "expected_strength": "WEAK",
        "expected_reasons": ["DEMOGRAPHIC_DOB_SELF"]
    },
    {
        "description": "Invalid MPIN (too short)",
        "dob": "01-01-1990",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "123",
        "expected_strength": "STRONG",
        "expected_reasons": []
    },
    {
        "description": "Invalid MPIN (non-digits)",
        "dob": "01-01-1990",
        "spouse_dob": "",
        "anniversary": "",
        "mpin": "abcd",
        "expected_strength": "STRONG",
        "expected_reasons": []
    }
]


In [64]:

def run_tests():
    print("⏳ Fetching live common PINs from Pocket-lint...")
    pins_4 = fetch_common_digit_pins_from_pocketlint("p", "Common four-digit PINs", 4)
    pins_6 = fetch_common_digit_pins_from_pocketlint("p", "Common six-digit PINs", 6)
    print("✅ Common PINs fetched successfully.\n")
    passed = 0
    for i, case in enumerate(test_cases, 1):
        dob = parse_date(case["dob"])
        spouse = parse_date(case["spouse_dob"]) if case["spouse_dob"] else None
        anniv = parse_date(case["anniversary"]) if case["anniversary"] else None
        mpin = case["mpin"]
        expected_strength, expected_reasons = case["expected_strength"],case["expected_reasons"]

        # Skip early if MPIN is invalid
        if not mpin.isdigit() or len(mpin) not in [4, 6]:
            strength, reasons = "STRONG", []
        else:
            is_common = mpin in pins_4 or mpin in pins_6
            demographics = {
                "dob_self": dob,
                "dob_spouse": spouse,
                "anniversary": anniv
            }
            demo_reasons = check_against_demographics(mpin, demographics)

            reasons = []
            if is_common:
                reasons.append("COMMONLY_USED")
            reasons.extend(demo_reasons)
            strength = "WEAK" if reasons else "STRONG"

        if strength == expected_strength and sorted(reasons) == sorted(expected_reasons):
            print(f"✅ Test {i}: {case['description']}")
            passed += 1
        else:
            print(f"❌ Test {i}: {case['description']}")
            print(f"   Expected: {expected_strength}, {expected_reasons}")
            print(f"   Got     : {strength}, {reasons}")

    print(f"\n📊 Summary: {passed}/{len(test_cases)} tests passed.")

if __name__ == "__main__":
    run_tests()

⏳ Fetching live common PINs from Pocket-lint...
✅ Common PINs fetched successfully.

✅ Test 1: Common 4-digit PIN
✅ Test 2: Common 6-digit PIN
✅ Test 3: MPIN from DOB (ddmm)
✅ Test 4: MPIN from DOB (mmyy)
✅ Test 5: MPIN from spouse DOB
✅ Test 6: MPIN from anniversary
✅ Test 7: 6-digit MPIN from DOB (ddmmyy)
✅ Test 8: 6-digit MPIN from anniversary (yymmdd)
✅ Test 9: Strong 4-digit MPIN
✅ Test 10: Strong 6-digit MPIN
✅ Test 11: Common + Demographic MPIN
✅ Test 12: Demographic from spouse date (ddyy)
✅ Test 13: Anniversary ddmmyy 6-digit
✅ Test 14: MPIN is full year of DOB
✅ Test 15: MPIN is reversed date
✅ Test 16: Empty spouse/anniv, random PIN
✅ Test 17: Just common 6-digit PIN
✅ Test 18: MMYY from DOB
✅ Test 19: Invalid MPIN (too short)
✅ Test 20: Invalid MPIN (non-digits)

📊 Summary: 20/20 tests passed.
